# Label and Feature Creation
In this notebook, I will import the single-column DataFrame of appellate opinions and create columns with labels and features. 
## TO DO:
<ul>
<li>0. Import Libraries 
<li>1. Clean up opinions - DONE
<ul><li>A. Remove page numbers from opinions
    <li>B. Remove Supreme Court case
    <li>C. Drop Dissents 
    </ul>
<li>2. Add file numbers - DONE 
<li>3. Create Labels section - DONE
<ul><li>A. Create function and apply to DataFrame 
<li>B. Clean up, combine, and drop columns
    <li>C: Address reversing-denials or affirming-denials </ul>
<li>4. Create case-type column -- DONE
<li>5. Extract Trial Court Judge -- PICK UP HERE -- CONTINUE REFINING JUDGE FUNCTION!!!!!!!!! - then drop
<li>6. Extract County -- NEED TO FIX "NONES" AND WRONG WORDS
<li>7. Topic Modeling Features - TBD
<li>8. EDA Visualizations - TBD
</ul>

1052 rows dropped for not fitting the affirmed/reversed criteria (errors, affirmed in part, appeals dismissed, etc.)<br>
38 rows dropped for not conforming to a reasonable case type or being too short to have a reasonable effect.<br>
84 rows dropped from the Supreme Court, reviewing underlying decisions (lacking the features of court of appeals cases)

## 0. Import Libraries and DataFrame

In [105]:
import io
import re
import pandas as pd
import pickle
import operator

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [106]:
# Open the dataframe
infile = open('ProjectData/df_clean.data', 'rb')
df = pickle.load(infile)
infile.close()

In [107]:
df.reset_index(inplace=True, drop=True)

In [108]:
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3922 entries, 0 to 3921
Data columns (total 1 columns):
Opinion    3922 non-null object
dtypes: object(1)
memory usage: 30.8+ KB


(None,                                              Opinion
 0   an unpublished opinion of the north carolina ...
 1  no. coa11-246 north carolina court of appeals ...
 2  no. coa08-347 north carolina court of appeals ...
 3  michael harrison gregory and wife, vivian greg...
 4  atlantic contracting and material company, inc...)

## 1. Remove Page Numbers & Supreme Court Cases

### A. Remove Page Numbers
Page numbers (-1-) appear throughout the corpus and complicated some of the following functions.  Accordingly, they're removed here. 

In [109]:
def remove_page_nums(string_):
    """
    A function to remove page numbers from a given string
    Page numbers must be the format "-##-"
    """
    new_string = re.sub(" -\d+- ", " ", string_)
    return(new_string)

In [110]:
no_page_list = []
for i in range(len(df.Opinion)):
    x = remove_page_nums(df.Opinion[i])
    no_page_list.append(x)
    
df["Opinion"] = pd.Series(no_page_list).values

In [111]:
df.head()

,Opinion
0,an unpublished opinion of the north carolina ...
1,no. coa11-246 north carolina court of appeals ...
2,no. coa08-347 north carolina court of appeals ...
3,"michael harrison gregory and wife, vivian greg..."
4,"atlantic contracting and material company, inc..."


### Fix Irish Names (O'xxx)

In [112]:
def lose_the_apostrophe(string_):
    """
    A function to remove apostrophes from Irish names; 
    The majority of the tokenization will be performed
    on the following dataframe.
    """
    new_string = re.sub(" o’|o'", " o", string_)
    
    return(new_string)

In [114]:
no_apostrophes = []
for i in range(len(df.Opinion)):
    x = lose_the_apostrophe(df.Opinion[i])
    no_apostrophes.append(x)
    
df["Opinion"] = pd.Series(no_apostrophes).values

### C. Remove Supreme Court Cases (that review Court of Appeals cases)
These cases review the Court of Appeals' decisions; they don't typically include features the same way that the Court of Appeals does, and their rulings are usually affecting the Court of Appeals (which may be the opposite of the underlying case). Accordingly, they should be dropped for this project. 

In [115]:
df['SCT'] = 0

In [116]:
# capture file number into new column
def drop_SCT(string_):
    sct_yes = re.search("in the supreme court of north carolina",string_[:50])
    if sct_yes == None:
        return(0)
    else:
        return(1)

In [117]:
df['SCT'] = df.Opinion.map(lambda x: drop_SCT(x))
len(df.loc[df['SCT'] == 1])

84

In [118]:
# drop supreme court cases
df.drop(list(df.loc[df['SCT'] == 1].index), inplace=True)
df.reset_index(inplace=True, drop=True)

### C. Trim Dissents, Because They are Not Relevant to the Function

#Dissents follow the opinion, when one of the three appellate judges doesn't agree; they may trigger further appeal, but they are not law, and may skew results, so they should be dropped. 

#### Exploration of opinions containing the phrase "dissenting"

In [119]:
dissents = df.loc[df['Opinion'].str.contains("dissenting")]

In [120]:
dissents.head()

,Opinion,SCT
13,"energy investors fund, l.p., inc., kvaerner as...",0
22,in the court of appeals of north carolina no....,0
27,"cecelia l. ford, administrator of the estate o...",0
35,"carolina place joint venture, plaintiff-appell...",0
45,christian emerson dysart and mildred maxwell d...,0


Review example opinions containing the word "dissent" to see which need to be dropped:</p>
Verbiage: (Y=actual dissent N=not a dissent)<br>
Y judge horton dissenting<br>
Y judge wynn dissents. ... wynn, judge, dissenting.<br>
Y tyson, judge dissenting.<br>
Y judge tyson concurs in part, dissents in part.<br>
Y calabria, judge, dissenting. // therefore, i respectfully dissent.<br>
Y justice martin dissenting. -- Conflicts w below -- maybe eliminate first 2000 chars? <br>
Y === wynn, judge dissenting.<br>
Y - elmore, judge, dissenting.<br>
Y. martin, chief judge, concurring in part, dissenting in part. - But later<br>
Y = campbell, judge, dissenting. i respectfully dissent <br>
Y == thomas, judge, dissenting.<br>
Y judge dillon dissents in a separate opinion. - 12,600 chars into 17,000 char opi<br>
<br>
N piazza v. kirkbride tyson, j., concurring in part, dissenting in part.<br>
N souter, j., dissenting (cit<br>
N r. justice martin dissenting. justice brady dissenting. a -- too early  (913 chars in, 80,000 char opinion)<br>
N (greene, j. dissenting) - case citation<br>
N . judge elmore dissenting. -- too early, 2400 chars in to 28,000 char opinion<br>
N m. chief judge martin concurring in part and dissenting in part. - 2731 chars into 29,000 char opinion<br>
N (hudson, j., dissenting) <br>
N   1715 chars into 23,000 char opinion<br>
------ never the (j., dissenting)<br>
<br>
Pseudocode:<br>
IF ends with "judges /2 and /2 concur." (not concurs) - there isn't a dissent (one of the two would be dissenting). <br>
ELSE if after the first 3,000 chars, of the opinion:<br>
    judge /4 (dissent*)<br>
    
#### Trimming-Dissents Function Creation and Revision
Created the below function following the preceding review; ran the function and reviewed the trimmed opinions for correctness (i.e., was something more or less than a dissent trimmed from the opinion).  The function was updated to put (judge|justice) in parens and to make the verb "dissents" instead of the noun "dissent", which was pulling references to other dissents. 


In [121]:
def trim_dissents(opinion):
    global trimmed_dissents
    if re.search("(dissent)", opinion):
        if re.search("judges\W+(?:\w+\W+){0,5}?concur\b", opinion):
            return()  
        else:
            try:
                y = re.search("(?:(judge|justice)\W+(?:\w+\W+){0,6}?dissents)", opinion[3000:])
                cut_off_point = y.start()
                opinion2 = opinion[:(cut_off_point+3000)]
                trimmed_dissents += 1
                return(opinion2)
            except:
                return(opinion)
    else:
        return(opinion)

In [122]:
# Drop dissents
trimmed_dissents = 0
new_opinions = []
for i in range(len(df.Opinion)):
    x = trim_dissents(df.Opinion[i])
    new_opinions.append(x)
    
# Replate opinions with trimmed opinions
df["Opinion"] = pd.Series(new_opinions).values

## 3. Creating a New Column for the File Numbers
The file numbers should have no predictive value; this section is more experiential than practical.

In [123]:
# capture file number into new column
def coa(string_):
    try:
        pat_coa_number = re.search("no.? ?coa.? ?(\d{2}-\d{1,5})",string_)
        return pat_coa_number.group(1)
    except:
        return('00-000')

In [124]:
coa_numbers = []
for i in range(len(df.Opinion)):
    x = coa(df.Opinion[i])
    coa_numbers.append(x)

df["File_Numbers"] = pd.Series(coa_numbers).values

In [125]:
df.head(5)

,Opinion,SCT,File_Numbers
0,an unpublished opinion of the north carolina ...,0,19-563
1,no. coa11-246 north carolina court of appeals ...,0,11-246
2,no. coa08-347 north carolina court of appeals ...,0,08-347
3,"michael harrison gregory and wife, vivian greg...",0,05-885
4,"atlantic contracting and material company, inc...",0,02-1087


## 3. Creating the Labels (Affirmed, Reversed, etc.)

The labels were created using the regex patterns below. The ultimate fuction below was created over many iterations. Initially, there were approximately 300 errors; the model was tweaked to reduce errors while maintaining reliability. Ultimately, 25 rows were dropped as errors because the cases were not beneficial to the model (i.e., they did not include a relevant summary judgment decision, the opinion on summary judgment was entwined with other components, etc.); 454 "affirmed-in-part" rows were dropped since they're not the binary outcome needed; and 408 dismissals were dropped since they don't have a usable outcome.  

### A. Functions to Assign and Apply Labels to the DataFrame

In [126]:
def labels(string_):
    """
    This function will extract the outcome from a given string (opinion).
    Each of the 'try' statements extracts the labels with decreasing 
    degrees of confidence (first by the one-word sentence, then within
    10 words of the statement regarding concurrence, typically near the 
    end of the opinion; then clipping the last 150 chars. of the opinion 
    and looking for the associated label-words, and finally looking for 
    label-words within 5-10 words of the often-occurring phrase, 'for the
    reasons set forth above').
    """
    try:
        try:
            try:
                try:  #this level has the highest confidence of getting an accurate label, based upon review of opinions (a single-word sentence)
                    labels = re.search("\.. ?(affirmed?)\.|\.?(reversed?)\.|(affirmed in part)|\.?(dismissed)\.",string_)
                    x = labels.group(1)
                    y = labels.group(2)
                    z = labels.group(3)
                    w = labels.group(4)
                    not_none = [x,y,z]
                    a = [i for i in not_none if i != None]
                    return a[0]
                except:  # slightly less confidence; looks for outcome word within 10 words of "concur", which frequently is at the end of a unanymous opinion
                    labels = re.search("(?:concurs?\W+(?:\w+\W+){0,40}?((affirmed in part)|reversed|affirmed|dismissed|no error|vacated)|((affirmed in part)|affirmed|reversed|dismissed|no error|vacated)\W+(?:\w+\W+){0,40}?concurs?)", string_)
                    #print("Group 0:", labels.group(0), "\nGroup 1:", labels.group(1), "\nGroup 2:", labels.group(2), "\nGroup 3:", labels.group(3), "\nGroup 4:", labels.group(4))
                    x = labels2.group(1)
                    y = labels2.group(2)
                    z = labels2.group(3)
                    w = labels2.group(4)
                    not_none = [x,y,z,w]
                    a = [i for i in not_none if i != None]
                    #print("This is resulting list a:", a)
                    return a[0]
            except: #slightly less confidence; if both of the previous methods fail, this clips the last 150 chars of the opinion for any of the outcome words
                clip = string_[-150:]
    #             print(clip)
                labels3 = re.search("('affirmed in part'|reversed|affirmed|dismissed|'affirm in part'|affirm|reverse|dismiss|improvidently allowed)",clip)
                return labels3.group(0)
        except: 
            labels4 = re.search("(?:reasons set forth?\W+(?:\w+\W+){0,5}?((affirm in part)|reverse|affirm|dismiss|no error|vacated?)|((affirm in part)|affirm|reversed?|dismiss|no error|vacated?)\W+(?:\w+\W+){0,10}?reasons set forth?)", string_)
            #print("Group 0:", labels.group(0), "\nGroup 1:", labels.group(1), "\nGroup 2:", labels.group(2), "\nGroup 3:", labels.group(3), "\nGroup 4:", labels.group(4))
            x = labels4.group(1)
            y = labels4.group(2)
            z = labels4.group(3)
            w = labels4.group(4)
            not_none = [x,y,z,w]
            a = [i for i in not_none if i != None]
            #print("This is resulting list a:", a)
            return a[0]
    except:
        return('error')

In [127]:
# Test Cell 
labels(df.Opinion[2092])

'error'

In [128]:
# Apply labels to the DataFrame
labels_list = []
for i in range(len(df.Opinion)):
    x = labels(df.Opinion[i])
    labels_list.append(x)

df["Result"] = pd.Series(labels_list).values

In [129]:
df.Result.value_counts()

affirmed                 2021
reversed                  558
affirmed in part          392
dismissed                 232
error                     200
dismiss                   190
reverse                   162
affirm                     75
improvidently allowed       6
no error                    1
vacate                      1
Name: Result, dtype: int64

### B. Combine Similar Terms and Drop Rows Irrelevant to the Outcome

In [130]:
df['Result'].replace(['reverse','affirm', 'dismiss','no error', 'vacated', 'improvidently allowed'],
                     ['reversed','affirmed','dismissed', 'affirmed', 'reversed', 'dismissed'], inplace=True)

# The model will treat 'no error' as 'affirmed' and 'vacated' as 'reversed'

In [131]:
df.Result.value_counts()

affirmed            2097
reversed             720
dismissed            428
affirmed in part     392
error                200
vacate                 1
Name: Result, dtype: int64

In [132]:
# Drop rows with 'error', 'dismissed', and 'affirmed in part' -- see section header, above
drop_list1 = df.loc[df['Result'] == 'error']
drop_list2 = df.loc[df['Result'] == 'affirmed in part']
drop_list3 = df.loc[df['Result'] == 'dismissed']
drop_list4 = df.loc[df['Result'] == 'vacate']

In [133]:
drop_list = list(drop_list1.index)+list(drop_list2.index)+list(drop_list3.index)+list(drop_list4.index)

In [134]:
df.drop(drop_list, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [135]:
df.Result.value_counts()

affirmed    2097
reversed     720
Name: Result, dtype: int64

### C. Ensure Labels Have the Same Effect Throughout the DataFrame (Substantial Right Appeals)

In [136]:
def substantial_right(string_):
    """ 
    I have been treating the label "affirm" as affirming the grant 
    of a summary judgment motion, and "reverse" as reversing the grant 
    of the motion, because it is much more common for the court of appeals 
    to address GRANTS of summary judgment. Much more rarely, they will 
    review a motion denying summary judgment, and the above functions will 
    have the opposite label than intended. This function will take a string, 
    review whether it contains the key words "substantial right," and then 
    analyze the language of the opinion to see if the opinion affirms or 
    reverses the DENIAL of summary judgment. If it does, it will return a
    '1', and if not, a '0'. This can be used later as a switch to flip the 
    label assigned above.
    """
    # Screen for the keword 'substantial right'
    if string_.count("substantial right") > 0:
        x = re.search("(?:(affirm|reverse?)\W+(?:\w+\W+){0,5}?denial)", string_)
        try:
            x.group(0)
            return(1)
        except:
            return(0)

In [137]:
substantial_right_list = []
for i in range(len(df.Opinion)):
    x = substantial_right(df.Opinion[i])
    substantial_right_list.append(x)

df["sub_right"] = pd.Series(substantial_right_list).values

In [138]:
df.loc[df['sub_right'] == 1]

,Opinion,SCT,File_Numbers,Result,sub_right
336,"estate of erik dominic williams, by and throug...",0,10-491,affirmed,1.0
357,no. coa11-1466 north carolina court of appeals...,0,11-1466,reversed,1.0
441,"estate of vera hewett, et al, plaintiffs, v. c...",0,08-1071,reversed,1.0
604,"james l. pierson, kathy l. pierson, lincoln m....",0,99-1333,affirmed,1.0
765,"mitchell, brewer, richardson, adams, burge & b...",0,09-1020,reversed,1.0
869,"nello l. teer company, inc., plaintiff, v. jon...",0,06-340,reversed,1.0
938,in the court of appeals of north carolina no....,0,16-908,reversed,1.0
957,an unpublished opinion of the north carolina c...,0,02-1610,affirmed,1.0
980,"wilson myers, administrator of the estate of t...",0,07-285,affirmed,1.0
995,"michael g. staley and melody h. staley, plaint...",0,98-1293,reversed,1.0


Review of the foregoing cases had so many with mixed issues that it will probably be better to drop all rows. 

In [139]:
indexes_to_drop = df.index[df['sub_right'] == 1].tolist()
df.drop(indexes_to_drop, inplace=True)

df.drop(['sub_right'], axis=1, inplace=True)

df.reset_index(inplace=True, drop=True)

## 4. Create Case-Type Feature By Sorting With Keywords

I created a simple sorting function which takes a dictionary of case types with associated keywords, and then it generates a popularity count of the various keywords, returning the highest-ranking case type for a given opinion. The dictionary was revised over many iterations and reviews; for instance, some words were not unique, were misleading, or needed leading/trailing spaces. 

In [140]:
# This dictionary contains types of law with associated, typically unique keywords
case_type_dict = {'premises':['premises', 'attractive nuisance', 'dangerous condition', 'slip and fall',
                              'defective condition', 'dog bite', 'landowner', 'vicious propensity',
                              'defective or unsafe condition', 'unsafe condition'], 
                  'products':['negligently manufactured', 'negligent manufacture', 'negligently designed',
                              'negligent design', 'manufacturing defect', 'products liability',
                              'product liability', 'ordinary use', 'product at issue', 'defective good'],
                  'car_crash':['collision', 'vehicle', 'motorist', 'other driver'], 
                  'corporate':['shareholder', 'bylaw', 'articles of incorporation', 'derivative', 
                               'corporate meetings', 'corporate books', 'annual meeting', 
                               'financial statements'],
                  'construction':['building defect', 'water intrusion', 'construction defect', 'building code',
                                  'subcontractor', 'construction industry'], 
                  'contract':[' formation', 'recission', 'specific performance', 'incidental damages',
                              'consequential damages', 'statute of frauds', 'complex business',
                              'contractual relationship', 'contract law', 'plain and unambiguous',
                              'promissory note', 'clinical privileges', 'agreement in effect', 
                              'breach of contract', 'terms of the agreement', 'default on the account',
                              'joint venture', 'credit account', 'payment of the debt', 'consignor',
                              'cosignee', 'oral contract', 'doctrine of necessaries', 'sold and delivered',
                              'defaulted on the loan', 'line of credit', 'credit card account', 
                              'security interest', 'past due and owing', 'contract price', 'guaranty agreement',
                              'contract to purchase', 'fdcpa'], 
                  'defamation':['libel', 'slander', 'defamatory', 'defamation'],
                  'employment':['wrongful discharge', 'discrimination', 'retaliation', 'retaliatory',
                                'retaliatory employment discrimination act', 'discriminatory',
                                'state personnel', 'state retirement plan', 'pay policy', 'salary',
                                'at-will employee', 'reinstatement', 'h-2a', 'wage and hour', 'fmla',
                                'disability benefits', 'employer-employee', 'wage laws'],
                  'estates':['intestate', 'probate', 'revocable trust', 'irrevocable trust', 'testator',
                             'holographic', 'residue', 'testate', 'partition and sale', 'undivided interest',
                             'executor', 'executrix', 'beneficiary', 'will conveying'],
                  'family_law':['divorce', 'custody', 'maintenance', 'child support', 'separation agreement',
                               'prenuptial', 'postnuptial', 'premarital', 'marital home', 'parental rights',
                               'alienation of affection', 'legally separated', 'marriage ceremony'], 
                  'fraud_udtpa':['unfair and deceptive', 'unfair or deceptive', 'chapter 75', 'fraud',
                                 'calculated to deceive'],
                  'governmental':['sovereign immunity', 'official capacity', 'governmental immunity', 
                                  'certificate of need', 'incarcerated', 'permit application', 
                                  'property tax commission', 'local school administrative unit',
                                  'public duty doctrine', 'malicious prosecution', 'inmate', 'wastewater',
                                  'separation of powers', 'secretary of revenue', 'school boards association',
                                  'department of environment and natural resources', 'municipal corporation',
                                  'local government retirement system'],
                  'insurance':['policy exclusion', 'insured party', 'coverage under the policy',
                               'insurance coverage'],
                  'landlord_tenant':['lease', 'landlord', 'security deposit', ' rent ', 'chapter 42', 'tenant'], 
                  'med_mal':['medical malpractice','health care profession', 'same or similar community', 
                             'rule 9(j)', 'rule 702(b)', 'continuing course of treatment', 'skilled nursing',
                             'care and treatment rendered'], 
                  'property':['easement', 'fee simple', 'tenants in common', 'joint tenants', 'nuisance', 
                             'eminent domain', 'escheat', 'replevin', 'zoning', 'mortgage', 'foreclosure',
                             'restriction agreement', 'reforming the deed', 'covenants and restrictions',
                             'property boundary', 'declaration of covenants', 'homeowners association',
                             'trespass to land', 'deed of trust', 'restrictive covenant', 'executed the deed',
                             'board of adjustment', 'improvement to land', 'quiet title', 'convey the property',
                             'blasting'], 
                  'workers_comp':['workers compensation', 'worker\'s compensation', 'fela',
                                  'workers\' compensation', 'workmen\'s compensation'],
                  'wrongful_death':['wrongful death']}

# 'dram_shop':['dram shop'] -- removed, only 2 that were not superseded by "car crash"

In [141]:
def case_type_sorter(dict_of_keywords, string):
    """ 
    This function takes a dictionary of case types and associated 
    keywords, assigns points for the frequency of the keywords
    of a given case type, and returns the case type with the highest
    number of points, as well as a confidence measure.  The dict_of_keywords 
    should be a dictionary of case-type keys and keyword values; the string 
    should be a single string.
    """
    counter_dict = {}

    # Iterate through dictionary, counting frequency of each keyword in the string/Opinion
    for key, values in dict_of_keywords.items():
        counter_dict[key] = 0
        for value in values:
            count = string.count(value)
            existing_count = counter_dict[key]
            counter_dict[key] = count + existing_count
    
    # Get total points for all keywords
    values = counter_dict.values()
    total_count = sum(values)
    
    # Ensure that if no case_types are matched, the type returned is 'other'
    if total_count > 0:
        likely_case_type = max(counter_dict.items(), key=operator.itemgetter(1))[0]
    else:
        likely_case_type = 'other'
    
    # Return a confidence ratio (points of most likely type / all points )
    try:
        confidence = str(round((counter_dict[likely_case_type]/total_count)*100,2))+'%'
    except:
        confidence = 'n/a'
    
    return likely_case_type, confidence

In [142]:
# Copy of the above function for case-by-case review
def case_type_test_sorter(dict_of_keywords, string):
    """ 
    This is a copy of the case_type_sorter function for case-by-case review
    """
    counter_dict = {}

    # Iterate through dictionary, counting frequency of each keyword in the string/Opinion
    for key, values in dict_of_keywords.items():
        counter_dict[key] = 0
        for value in values:
            count = string.count(value)
            existing_count = counter_dict[key]
            counter_dict[key] = count + existing_count
            
    return counter_dict

In [143]:
# Apply case_type and confidence level columns to the DataFrame

case_type_list = []
case_type_confidence = []
for i in range(len(df.Opinion)):
    y,z = case_type_sorter(case_type_dict, df.Opinion[i])
    case_type_list.append(y)
    case_type_confidence.append(z)

df["Case_Type"] = pd.Series(case_type_list).values
df["Case_Type_Confidence"] = pd.Series(case_type_confidence).values

In [144]:
one = df.loc[(df['Case_Type'] == 'other') & (df.Opinion.str.len() > 1350)]

In [145]:
#remaining "other" case types and cases shorter than 1350 characters (lacking any type information)
len(one)

39

In [146]:
# Test function to review cases and diagnose issues (ultimately to update the case_type_dict)
try:
    test_cell = int(input("Index to Test:"))
    print(case_type_sorter(case_type_dict, df.Opinion[test_cell]),
          '\n\n', case_type_test_sorter(case_type_dict, df.Opinion[test_cell]),
          '\n\n', df.Opinion[test_cell])
except:
    print("Please enter an index to review:")

Index to Test:
Please enter an index to review:


In [147]:
drop_list_others = list(one.index)
df.drop(drop_list_others, inplace=True)
df.reset_index(drop=True, inplace=True)

In [148]:
df.Case_Type.value_counts()

property           369
contract           333
car_crash          318
fraud_udtpa        275
governmental       246
landlord_tenant    229
family_law         187
premises           162
employment         144
estates            125
med_mal             88
corporate           77
construction        51
defamation          44
insurance           27
wrongful_death      23
other               18
products            17
workers_comp        14
Name: Case_Type, dtype: int64

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 6 columns):
Opinion                 2747 non-null object
SCT                     2747 non-null int64
File_Numbers            2747 non-null object
Result                  2747 non-null object
Case_Type               2747 non-null object
Case_Type_Confidence    2747 non-null object
dtypes: int64(1), object(5)
memory usage: 128.9+ KB


## 5.  Extract Feature: Trial-Court Judge

In [163]:
def trial_judge(opinion):
    """
    This function extracts the underlying trial judge
    from the opinion, typically falling between the 
    words, 'by judge ____ in'
    """
    try:
        try:
            try:
                try:
                    judge = re.search("by judge (\w+) (\w+) in", opinion)
                    f_name = judge.group(1)
                    l_name = judge.group(2)
                    pre_clean = [l_name, f_name]
                    name = [i for i in pre_clean if i != None]
                    return(name)
                except:
                    judge = re.search("by (the honorable|judge) (\w+.?) ?(\w+.|-?)? ?(\w+),? ?(jr.|sr.|iii|ii|iv)?,? (?:in|and)", opinion)
                    f_name = judge.group(2)
                    m_name = judge.group(3)
                    l_name = judge.group(4)
                    suffx =  judge.group(5) 
                    pre_clean = [l_name, f_name, m_name, suffx]
                    name = [i for i in pre_clean if i != None]
                    return(name)
            except:
                judge = re.search("by (\w+.?), .?\.? ?j., in", opinion)
                l_name = judge.group(1)
                pre_clean = [l_name]
                name = [i for i in pre_clean if i != None]
                return(name)
        except:
            judge = re.search("(the honorable|judge) (\w+.?) (\w+.|-?)? ?(\w+),? ?(jr.|sr.|iii|ii|iv)?,? (?:presiding|in|on)", opinion)
            f_name = judge.group(2)
            m_name = judge.group(3)
            l_name = judge.group(4)
            suffx =  judge.group(5) 
            pre_clean = [l_name, f_name, m_name, suffx]
            name = [i for i in pre_clean if i != None]
            return(name)
    except:
        name = 'error'
        return(name)
    

In [164]:
# Apply function to the DataFrame
judges_list = []
for i in range(len(df.Opinion)):
    y = trial_judge(df.Opinion[i])
    judges_list.append(y)
    
df["Trial_Judge"] = pd.Series(judges_list).values

In [165]:
df.head()

,Opinion,SCT,File_Numbers,Result,Case_Type,Case_Type_Confidence,Trial_Judge
0,an unpublished opinion of the north carolina ...,0,19-563,affirmed,property,84.62%,"[williams, karen , eady-]"
1,no. coa11-246 north carolina court of appeals ...,0,11-246,affirmed,employment,55.26%,"[hight, henry , w., jr.]"
2,no. coa08-347 north carolina court of appeals ...,0,08-347,affirmed,property,85.71%,"[poovey, nathaniel , j.]"
3,"michael harrison gregory and wife, vivian greg...",0,05-885,affirmed,car_crash,33.33%,"[gore, william , c.]"
4,in the court of appeals of north carolina no....,0,17-112,reversed,governmental,100.0%,"[stephens, donald , w.]"


In [166]:
x = df.loc[df["Trial_Judge"] == 'error']
print("Num_Errors:",len(x), "\n", x)

Num_Errors: 80 
                                                 Opinion  SCT File_Numbers    Result  \
30    hospice & palliative care charlotte region d/b...    0      06-1484  affirmed   
35    no. coa11-868 north carolina court of appeals ...    0       11-868  affirmed   
162   no. coa07-54 north carolina court of appeals f...    0        07-54  affirmed   
204   gary anderson smith, plaintiff, v. brian oneal...    0       06-688  reversed   
215   no. coa02-13 north carolina court of appeals f...    0        02-13  affirmed   
246   j. kamau christopher a/k/a joseph kamau christ...    0      07-1516  affirmed   
269   james e. long, commissioner of insurance of no...    0       03-638  affirmed   
382   an unpublished opinion of the north carolina c...    0       07-240  affirmed   
452   durham land owners association, an unincorpora...    0       05-736  reversed   
471   an unpublished opinion of the north carolina c...    0      14-1196  affirmed   
493   an unpublished opini

In [167]:
# Test function to review and diagnose issues
try:
    test_cell = int(input("Index to Test:"))
    print(trial_judge(df.Opinion[test_cell]),
          '\n\n', df.Opinion[test_cell])
except:
    print("Please enter an index to review:")

Index to Test:779
error 

 no. coa14-160 north carolina court of appeals filed: 4 november 2014 cumberland county hospital system, inc. d/b/a cape fear valley health system, petitioner, v. nc department of health and human services, division of health service regulation, certificate of need section, respondent, and firsthealth of the carolinas, inc., respondent-intervenor. from the office of administrative hearing (cumberland county) no.12 dhr 12094 appeal by petitioner from final decision entered 17 september 2013 by administrative law judge beecher r. gray. heard in the court of appeals 14 august 2014. k&l gates llp, by gary s. qualls, susan k. hackney and steven g. pine for petitioner-appellant. nelson mullins riley & scarborough llp, by noah h. huffstetler, iii, denise m. gunter, and candace s. friel, for respondent/intervenor-appellee firsthealth. attorney general roy cooper, by special deputy attorney general june s. ferrell and assistant attorney general scott t. stroud for resp

In [ ]:
len( "harland dean campbell v. city of high point no. 438a01 (filed 18 december 2001) appeal pursuant to n.c.g.s. § 7a-30(2) from the decision of a divided panel of the court of appeals, ___ n.c. app. ___, 551 s.e.2d 443 (2001), affirming an order for summary judgment signed 4 april 2000 by albright, j., in superior court, guilford county. heard in the supreme court 10 december 2001. skager law firm, by philip r. skager, for plaintiff- appellant. womble ca")

162 - multiple
204 - multiple
215 - by judge clarence w. carter, superior court -- BY JUDGE ... but no IN
246 - nonjudicial
269 - nonjudicial
285 - 1stimpression
382 - multiple
452 - multiple
471 - nonjudicial
493 - order entered 23 january 2003 by danny e. davis in  -- BY but no JUDGE ... IN
496 - nonjudicial
521 - by judge jack a. thompson at the -- "AT" instead of "IN"
576 - nonjudicial
606 - multiple
664 - multiple
670 - 1997 by w. russell duke, jr. in -- BY but no JUDGE ... IN 2222222222222222222
715 - nonjudicial
722 - SCT
779 - by administrative law judge beecher r. gray.

## 7. Extract Feature: County

In [ ]:
def extract_county(text):
    try:
        county = re.search("((\w*) county)",text[:700])
        return county.group(2)
    except:
        return(None)

In [ ]:
extract_county(df.Opinion[200])

In [ ]:
county_list = []
for i in range(len(df.Opinion)):
    x = extract_county(df.Opinion[i])
    county_list.append(x)
    
df["County"] = pd.Series(county_list).values

# Work through list below - resolve "None" and wrong counties

In [ ]:
list = df.County.value_counts()
print(list)

## 8. Topic Modeling/One Shot Learning to Extract Embedded Features

## 9. EDA Visualizations

### A. MSJ Allowed/Reversed Over Time

### B. MSJ by Case Type